# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234211192012                   -0.50    7.0         
  2   -7.249858111261       -1.81       -1.39    1.0   10.7ms
  3   -7.250950812143       -2.96       -1.81    2.0   13.3ms
  4   -7.251035258326       -4.07       -1.92    1.0   10.7ms
  5   -7.251316543840       -3.55       -2.48    1.0   11.5ms
  6   -7.251337748898       -4.67       -3.24    1.0   11.3ms
  7   -7.251338737887       -6.00       -3.55    2.0   14.1ms
  8   -7.251338786528       -7.31       -3.97    1.0   11.6ms
  9   -7.251338794942       -8.08       -4.27    2.0   13.7ms
 10   -7.251338798444       -8.46       -4.82    1.0   12.2ms
 11   -7.251338798652       -9.68       -5.07    2.0   91.4ms
 12   -7.251338798694      -10.37       -5.49    1.0   13.2ms
 13   -7.251338798703      -11.03       -6.05    2.0   14.2ms
 14   -7.251338798704      -11.96       -6.41    2.0   14.9ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.08456233519723523
[ Info: Arnoldi iteration step 2: normres = 0.5179755840173649
[ Info: Arnoldi iteration step 3: normres = 0.9022383685726841
[ Info: Arnoldi iteration step 4: normres = 0.29945840114044103
[ Info: Arnoldi iteration step 5: normres = 0.4327932321046853
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.27e-02, 5.34e-02, 3.05e-01, 2.97e-01, 4.61e-02)
[ Info: Arnoldi iteration step 6: normres = 0.2799077910190465
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (7.71e-03, 6.63e-02, 2.52e-01, 7.51e-02, 2.96e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0684593419020992
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.54e-04, 5.23e-03, 1.46e-02, 2.18e-02, 5.73e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10264934506905754
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.10e-0